In [27]:
export SUFFIX=$(date +%s | cut -c6-10)
export RESOURCE_GROUP="rg-aio-$SUFFIX"
export CLUSTER_NAME="mycluster"
export LOCATION="westus2"
export APP_NAME="sp-aio-$SUFFIX"

In [ ]:
az login &

> If you are in a GitHub Codespace and cannot use `--device-code` it will redirected to a "localhost" url. Copy the url and replace it in the next cell.

In [ ]:
curl '<URL>'

In [5]:
# Id of the subscription where your resource group and Arc-enabled cluster will be created

SUBSCRIPTION_ID=$(az account show --query id -o tsv)
az account set -s $SUBSCRIPTION_ID

In [ ]:
./provisioning/00-clean-up.sh

In [ ]:
az ad sp create-for-rbac --name $APP_NAME --role owner --scopes /subscriptions/$SUBSCRIPTION_ID --json-auth

In [19]:
pushd ../deploy
cp ./sample-aio.auto.tfvars.example ./$CLUSTER_NAME.auto.tfvars

cat << EOF >> $CLUSTER_NAME.auto.tfvars
resource_group_name          = "$RESOURCE_GROUP"
arc_cluster_name             = "$CLUSTER_NAME"
key_vault_name               = "$KEYVAULT_NAME"
EOF

sed -i 's/vm_computer_name\s*=\s*".*"/vm_computer_name             = "vm-'$CLUSTER_NAME'"/' $CLUSTER_NAME.auto.tfvars
sed -i 's/vm_username\s*=\s*".*"/vm_username                  = "admin'$CLUSTER_NAME'"/' $CLUSTER_NAME.auto.tfvars
sed -i 's/vm_ssh_pub_key_file\s*=\s*".*"/vm_ssh_pub_key_file          = "~/.ssh/id_aio_rsa.pub/' $CLUSTER_NAME.auto.tfvars
popd

/workspaces/azure-edge-extensions-aio-iac-terraform/deploy /workspaces/azure-edge-extensions-aio-iac-terraform/aio


/workspaces/azure-edge-extensions-aio-iac-terraform/aio


In [ ]:
# Get the list of resource groups
resource_groups=$(az group list --query "[].name" -o tsv)

# Loop through each resource group
for rg in $resource_groups
do
    echo "Processing resource group: $rg"
    
    # Run the clean-up script
    az group delete --name $rg --yes --no-wait
done